Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


Считываем файл

In [205]:
data = pd.read_excel('test.xlsx')
data.head(10)


,Provider ID,Operation ID,Payment ID,Operation Type,Operation Status,Secure 3ds Check,Channel Amount,Channel Currency,Operation created at Date,Payment Method Type,Account Number,Merchant ID,Project ID,Customer ID,Customer Email,Card Holder,Issuer Country Name,Country By IP Name,Customer IP V4 V6,Issuer Bank Name
0,4781.0,99892010097211,6021093719,sale,success,1.0,1000.0,RUB,2022-03-30,visa,427626******5062,39,321,3753071,avtomr@yandex.ru,LEONID PROKHOROV,Russian Federation,Russian Federation,176.59.134.184,SBERBANK of Russia
1,1293.0,99888010096111,347788572,sale,success,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
2,1293.0,99888010096101,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
3,1293.0,99888010096091,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
4,1293.0,99888010096081,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
5,1293.0,99888010096071,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
6,1293.0,99888010096061,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
7,1293.0,99888010096051,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
8,1293.0,99888010096041,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S
9,1293.0,99888010096031,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,434597******0730,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S


In [202]:
data.isnull().sum()



Provider ID                    355
Operation ID                     0
Payment ID                       0
Operation Type                   0
Operation Status                 0
Secure 3ds Check             12366
Channel Amount                   3
Channel Currency                13
Operation created at Date        0
Payment Method Type              7
Account Number                 375
Merchant ID                      0
Project ID                       0
Customer ID                     33
Customer Email                1495
Card Holder                   1151
Issuer Country Name           1062
Country By IP Name              56
Customer IP V4 V6                0
Issuer Bank Name              1176
dtype: int64

In [214]:
data_has_null = data.drop(columns=['Secure 3ds Check', 'Customer Email'])
data_has_null = data_has_null[data_has_null.isnull().any(axis=1)]
agg_na = data_has_null.groupby(['Merchant ID', 'Project ID']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_customers=('Customer ID', 'nunique'),
    unique_IPs=('Customer IP V4 V6', 'nunique'),
    unique_Operations=('Operation ID', 'nunique'),
    
    unique_acc_number=('Account Number', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    total_country_missmatch=('Country By IP Name', lambda x: (x != data.loc[x.index, 'Issuer Country Name']).sum()),
).reset_index()
agg_na = agg_na.sort_values(by='total', ascending=False)
# .div(agg_projects['total'], axis=0).mul(100).round()
# try:
#     agg_projects[columns_to_convert] = agg_projects[columns_to_convert].applymap(lambda x: f'{int(x)}%')
# except:
#     pass

agg_na

,Merchant ID,Project ID,total,total_decline,unique_customers,unique_IPs,unique_Operations,unique_acc_number,unique_cardholders,total_country_missmatch
7,278,983,475,146,443,440,475,410,3,414
2,46,433,326,252,137,161,326,72,45,304
0,39,321,326,115,106,121,326,102,12,229
3,212,726,175,175,170,2,175,169,1,175
1,46,162,123,59,20,22,123,0,0,123
11,1581,5771,89,88,4,6,89,80,87,88
10,300,1098,71,38,56,58,71,57,0,42
8,278,22431,17,17,1,4,17,15,1,16
4,212,5591,9,9,1,2,9,1,1,0
6,212,30791,8,6,4,4,8,4,4,8


In [233]:
agg_data = data.groupby(['Card Holder']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_acc_number=('Account Number', 'nunique'),
    unique_ips=('Customer IP V4 V6', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique')
).reset_index()
agg_data = agg_data.sort_values(by='unique_acc_number', ascending=False)

fraud_lst = ['ALEKSEY SHISHKIN', 'JOHN DOE' , 'NO NAME']
data_fraud1 = data[data['Card Holder'].isin(fraud_lst)]
data_fraud2 = data[data['Customer IP V4 V6'] == '127.0.0.1']
agg_data.head(10)

,Card Holder,total,total_decline,unique_acc_number,unique_ips,unique_cardholders,unique_emails
269,ALEKSEY SHISHKIN,202,202,184,4,1,1
1909,JOHN DOE,277,111,155,170,1,2
2594,NO NAME,36,36,17,5,1,1
1989,KAPITAL BANK,28,20,11,13,1,1
2422,MOMENTUM R,13,6,6,6,1,6
3862,VISA,13,10,5,5,1,1
2382,MICHAEL ONWURAH,23,20,4,16,1,1
2689,OLEGARIO RODRIGUEZ,10,10,4,3,1,2
666,AZEEZ ODUTOLA,4,3,3,2,1,1
695,BAHRUZ ISMAYILOV,7,7,3,1,1,1


In [235]:
agg_data = data.groupby(['Customer IP V4 V6']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_acc_number=('Account Number', 'nunique'),
    unique_countries=('Country By IP Name', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique')
).reset_index()
agg_data = agg_data.sort_values(by='unique_acc_number', ascending=False)
fraud_lst2 = agg_data.head(20)['Customer IP V4 V6']
data_fraud3 = data[data['Card Holder'].isin(fraud_lst2)]
agg_data.head(20)

,Customer IP V4 V6,total,total_decline,unique_acc_number,unique_countries,unique_cardholders,unique_emails
3260,5.62.19.48,242,242,192,1,195,1
3258,5.62.18.88,123,123,112,1,1,1
3257,5.62.18.60,75,75,69,1,1,1
4767,95.174.65.147,174,171,55,1,60,1
3617,80.255.7.20,33,33,33,1,33,1
3264,5.62.62.185,42,42,28,1,28,1
3263,5.62.60.194,49,49,26,1,24,1
3614,80.255.13.117,29,29,21,1,21,1
3262,5.62.58.121,31,31,15,1,15,1
3768,83.142.83.20,29,29,13,1,13,1


In [105]:
data_bank_missing = data[data['Issuer Country Name'].isna()]
missed_merch = data_bank_missing['Merchant ID'].unique().tolist()
agg_data = data_bank_missing.groupby(['Merchant ID', 'Project ID']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_acc_number=('Account Number', 'nunique'),
    unique_countries=('Country By IP Name', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique')
).reset_index()
agg_data = agg_data.sort_values(by='total', ascending=False)
agg_data.head()



,Merchant ID,Project ID,total,total_decline,unique_acc_number,unique_countries,unique_cardholders,unique_emails
3,278,983,414,112,361,3,0,351
2,46,433,263,193,26,40,1,108
0,39,321,218,23,90,13,0,18
1,46,162,123,59,0,4,0,20
4,300,1098,42,10,41,2,0,0


In [189]:
data_bank_missing = data[data['Issuer Country Name'].isna()]
data_bank_missing.head(10)

,Provider ID,Operation ID,Payment ID,Operation Type,Operation Status,Secure 3ds Check,Channel Amount,Channel Currency,Operation created at Date,Payment Method Type,Account Number,Merchant ID,Project ID,Customer ID,Customer Email,Card Holder,Issuer Country Name,Country By IP Name,Customer IP V4 V6,Issuer Bank Name
18,1083.0,99817010094891,299486,sale,decline,NaN,17.70,USD,2022-02-27,webmoney-light,NaN,46,433,355878,chemutairuth20@gmail.com,NaN,NaN,Kenya,105.48.159.220,NaN
23,1178.0,9967010044951,292536,sale,decline,NaN,200.00,MYR,2022-02-11,banks/malaysia,NaN,46,162,284261,joelai90@yahoo.com,NaN,NaN,Malaysia,202.186.143.202,NaN
46,2791.0,9938010044521,294668,sale,decline,NaN,600.00,THB,2022-02-17,banks/thailand,NaN,46,433,352624,kolin0949281008@gmail.com,NaN,NaN,Thailand,1.47.27.183,NaN
130,2801.0,98220010095101,294667,sale,decline,NaN,600.00,THB,2022-02-17,banks/thai-qr,NaN,46,433,352624,kolin0949281008@gmail.com,NaN,NaN,Thailand,1.47.27.183,NaN
163,3211.0,97217010094201,292068,sale,decline,NaN,100.00,MYR,2022-02-10,banks/malaysia,NaN,46,162,284261,joelai90@yahoo.com,NaN,NaN,Malaysia,202.186.143.202,NaN
165,1521.0,97124010096841,6021018333,sale,success,NaN,20000.00,UZS,2022-03-30,card-partner,1521-703cd2530dc5d1ffa84a006bf39b5a1e,39,321,4639134,NaN,NaN,NaN,Uzbekistan,188.113.216.215,NaN
191,1521.0,96443010098511,6021159272,sale,success,NaN,100000.00,UZS,2022-03-30,card-partner,1521-487fa1193844a877d70757192afa6aa9,39,321,4054450,NaN,NaN,NaN,Uzbekistan,185.139.137.179,NaN
203,1083.0,95968010095941,298344,sale,decline,NaN,168.17,USD,2022-02-25,webmoney-light,NaN,46,433,289753,torben_schaefer@hotmail.de,NaN,NaN,Germany,134.101.158.211,NaN
216,1083.0,95520010094401,294728,sale,decline,NaN,10.00,USD,2022-02-17,webmoney-light,NaN,46,433,352693,dellingsalpha@gmail.com,NaN,NaN,Malawi,41.70.59.62,NaN
230,3211.0,95051010094091,294221,sale,decline,NaN,100.00,MYR,2022-02-15,banks/malaysia,NaN,46,162,352003,zulinah1998@gmail.com,NaN,NaN,Malaysia,113.210.102.91,NaN


In [104]:
data[data['Merchant ID'].isin(missed_merch)]['Project ID'].unique().tolist()

[321, 5771, 433, 162, 22431, 26751, 983, 1098, 5741, 5761]

In [167]:

agg = data.groupby(['Payment Method Type']).size().reset_index(name='counts')

counts_sorted = agg.sort_values(by='counts', ascending=False, inplace=True)
agg.head(10)

,Payment Method Type,counts
34,visa,8811
28,mastercard,3396
29,mir,269
2,BEELINE,198
5,TELE2,135
14,banks/malaysia,133
39,webmoney-light,128
20,card-partner,94
31,monetix,81
9,bank-transfer/upi,64


In [154]:
agg_merchants = data.groupby('Merchant ID').agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_IPs=('Customer IP V4 V6', 'nunique'),
    unique_acc_number=('Account Number', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique'),
    total_country_missmatch=('Country By IP Name', lambda x: (x != data.loc[x.index, 'Issuer Country Name']).sum())
).reset_index()
agg_merchants = agg_merchants.sort_values(by='total', ascending=False)

columns_to_convert = agg_merchants.columns.difference(['Merchant ID', 'total'])

agg_merchants[columns_to_convert] = agg_merchants[columns_to_convert]
# .div(agg_merchants['total'], axis=0).mul(100).round()
# agg_merchants[columns_to_convert] = agg_merchants[columns_to_convert].applymap(lambda x: f'{int(x)}%')

agg_merchants.head(10)



,Merchant ID,total,total_decline,unique_IPs,unique_acc_number,unique_cardholders,unique_emails,total_country_missmatch
5,1581,2947,2690,231,468,473,173,762
1,46,2795,1874,923,656,599,618,900
2,212,2675,1714,1118,1258,958,109,238
0,39,2500,1202,952,853,744,753,456
3,278,2438,1378,1757,1715,1331,1048,445
4,300,159,121,75,103,45,5,113
8,6841,36,36,3,36,34,1,35
6,2901,22,22,1,12,1,1,22
7,4011,3,3,1,2,1,1,2


In [217]:
agg_projects = data.groupby(['Merchant ID', 'Project ID']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_customers=('Customer ID', 'nunique'),
    unique_IPs=('Customer IP V4 V6', 'nunique'),
    unique_Operations=('Operation ID', 'nunique'),
    
    unique_acc_number=('Account Number', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique'),
    total_country_missmatch=('Country By IP Name', lambda x: (x != data.loc[x.index, 'Issuer Country Name']).sum()),
    total_3ds=('Secure 3ds Check', lambda x: (x == 1.0).sum()),
).reset_index()
agg_projects = agg_projects.sort_values(by='total', ascending=False)

columns_to_convert = agg_merchants.columns.difference(['Merchant ID', 'total'])

agg_projects[columns_to_convert] = agg_projects[columns_to_convert]
# .div(agg_projects['total'], axis=0).mul(100).round()
# try:
#     agg_projects[columns_to_convert] = agg_projects[columns_to_convert].applymap(lambda x: f'{int(x)}%')
# except:
#     pass

agg_projects.head(20)


,Merchant ID,Project ID,total,total_decline,unique_customers,unique_IPs,unique_Operations,unique_acc_number,unique_cardholders,unique_emails,total_country_missmatch,total_3ds
25,1581,5771,2877,2642,162,216,2877,455,461,162,747,0
2,46,433,2672,1815,616,909,2672,656,599,606,777,0
0,39,321,2500,1202,828,952,2500,853,744,753,456,894
11,212,12571,2013,1311,837,833,2013,814,831,1,29,0
19,278,983,1547,1022,1100,1104,1547,1071,671,351,414,0
20,278,22431,688,202,580,583,688,583,580,580,21,220
13,212,27361,231,79,146,153,231,138,1,1,8,0
21,278,26751,203,154,121,120,203,128,120,121,10,41
3,212,726,177,177,172,4,177,171,3,3,175,0
22,300,1098,159,121,70,75,159,103,45,5,113,0


In [177]:
def filter_users_more_than_once(df, user_column):
    user_counts = df[user_column].value_counts()
    users_to_keep = user_counts[user_counts == 1].index
    filtered_df = df[df[user_column].isin(users_to_keep)]

    return filtered_df

data_5771 = data[data['Project ID'] == 5771]
data_5771_filtered = filter_users_more_than_once(data_5771, 'Customer ID')
len(data_5771_filtered)


28

# Что стоит отметить из аномального:
* у проекта 726 все операции отклонились, при этом на 171 номер аккаунта всего 4 IP адреса. Выглядит как мошенничнские действия.
* аналогично у проекта 18981 все операции были отклонены, при этом номеров карты 12, а уникальных ip, имен кардхолдера и емаилв всего один
* у самого большого проекта 5771 очнь высокая доля операции типа decline по сравнению с остальными, мало уникальных ip адресов, кардхолдеров и емайлов

In [134]:
agg = data.groupby('Customer IP V4 V6')['Customer ID'].nunique().reset_index(name='counts')

counts_sorted = agg.sort_values(by='counts', ascending=False, inplace=True)
agg.head(10)


,Customer IP V4 V6,counts
3258,5.62.18.88,104
3257,5.62.18.60,69
2567,45.45.45.45,8
5032,95.86.178.234,6
3115,5.44.37.187,5
3947,85.132.106.196,5
1178,185.32.44.98,4
3006,5.197.226.121,4
3203,5.44.39.167,4
1176,185.32.44.122,3


In [215]:
agg = data.groupby('Operation Type').size().reset_index(name='counts')

counts_sorted = agg.sort_values(by='counts', ascending=False, inplace=True)
agg.head(10)

,Operation Type,counts
6,sale,11317
3,payout,2199
4,recurring,26
0,commission,19
1,customer action,10
2,manual change,3
5,refund,1


In [114]:
data[(data['Operation Type'] != 'payout') & (data['Operation Type'] != 'sale')]

,Provider ID,Operation ID,Payment ID,Operation Type,Operation Status,Secure 3ds Check,Channel Amount,Channel Currency,Operation created at Date,Payment Method Type,Account Number,Merchant ID,Project ID,Customer ID,Customer Email,Card Holder,Issuer Country Name,Country By IP Name,Customer IP V4 V6,Issuer Bank Name
240,4971.0,94739010097691,2521af6bd6fe2da127d24b75017c9c82,recurring,success,NaN,3.00,AZN,2022-03-22,mastercard,510307******5341,278,22431,198a7a69b510b4d85208b06883d5692a,dadasovtelman1@gmail.com,TELMAN DADASOV,Azerbaijan,Azerbaijan,5.191.19.65,KAPITAL BANK
258,1095.0,93885010093551,capitaller-transfer-b0462e3ce0a6e03d38a8614ade,commission,success,NaN,1.86,USD,2022-02-04,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
296,1095.0,92558010096161,capitaller-transfer-111e2bc075f4fc5035d80ddc53,commission,success,NaN,0.42,USD,2022-02-22,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
376,1095.0,90789010096781,capitaller-transfer-8e0d62ada09bbe339329f50a03,commission,success,NaN,1.23,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
549,4971.0,87364010097591,4ae4ad6a0322868aac73af8f57a65393,recurring,success,NaN,4.00,AZN,2022-03-22,visa,412720******1260,278,22431,764a493502a004a74a56eeadf8909f66,jalismayilova@gmail.com,JAL ISMAYILOVA,Azerbaijan,Azerbaijan,212.47.143.59,International Bank of Azerbaijan
810,1421.0,82975010097781,D318990709,customer action,success,NaN,0.00,NaN,2022-03-22,mobile,NaN,278,983,10275580,pakanela20@gmail.com,NaN,NaN,Kazakhstan,81.88.148.19,NaN
971,4971.0,80605010097001,f0e2c71d82cdc1c7c2370e74c139b852,recurring,decline,NaN,17.00,AZN,2022-03-22,mastercard,526163******9157,278,22431,468b02fe0af6f1bde3788d0ad49de277,pervinmahmudov6@gmail.com,NIZAMI LIYEV,Azerbaijan,Azerbaijan,185.26.186.15,RABITABANK OJSC
1010,4971.0,80316010097281,008cd4eb0082c9480d850d06138db736,recurring,success,NaN,15.00,AZN,2022-03-22,visa,416973******0952,278,22431,2d3cdd9c3380b2eb5be68aa51bf87170,ulkerparlanova@gmail.com,ULKER KAMALLI,Azerbaijan,Azerbaijan,194.135.166.112,Kapital Bank JSB
1144,1095.0,78449010096471,capitaller-transfer-77b17ef6ada45d43cece7705ac,commission,success,NaN,1.24,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
1170,4971.0,78118010096751,667c73b7ebcfe67de0f21ca4f5fabe67,recurring,decline,NaN,6.00,AZN,2022-03-22,visa,416974******1812,278,22431,3af61ac5af72621dc57a4187b95b06cf,aljevramil@gmail.com,ALIYEV RAMIL,Azerbaijan,Azerbaijan,82.194.20.99,Kapital Bank JSB


In [116]:
data[data['Customer IP V4 V6'] == '127.0.0.1']

,Provider ID,Operation ID,Payment ID,Operation Type,Operation Status,Secure 3ds Check,Channel Amount,Channel Currency,Operation created at Date,Payment Method Type,Account Number,Merchant ID,Project ID,Customer ID,Customer Email,Card Holder,Issuer Country Name,Country By IP Name,Customer IP V4 V6,Issuer Bank Name
258,1095.0,93885010093551,capitaller-transfer-b0462e3ce0a6e03d38a8614ade,commission,success,NaN,1.86,USD,2022-02-04,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
259,1095.0,93885010093541,capitaller-transfer-b0462e3ce0a6e03d38a8614ade,payout,success,NaN,44.64,USD,2022-02-04,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
296,1095.0,92558010096161,capitaller-transfer-111e2bc075f4fc5035d80ddc53,commission,success,NaN,0.42,USD,2022-02-22,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
297,1095.0,92558010096151,capitaller-transfer-111e2bc075f4fc5035d80ddc53,payout,success,NaN,10.09,USD,2022-02-22,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
376,1095.0,90789010096781,capitaller-transfer-8e0d62ada09bbe339329f50a03,commission,success,NaN,1.23,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
377,1095.0,90789010096771,capitaller-transfer-8e0d62ada09bbe339329f50a03,payout,success,NaN,29.58,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
1144,1095.0,78449010096471,capitaller-transfer-77b17ef6ada45d43cece7705ac,commission,success,NaN,1.24,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
1145,1095.0,78449010096461,capitaller-transfer-77b17ef6ada45d43cece7705ac,payout,success,NaN,29.72,USD,2022-03-03,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
1530,1095.0,73291010095231,capitaller-transfer-5bb0abf577b1e626e2beb4f6fc,commission,success,NaN,1.82,USD,2022-02-21,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN
1531,1095.0,73291010095221,capitaller-transfer-5bb0abf577b1e626e2beb4f6fc,payout,success,NaN,43.75,USD,2022-02-21,capitaller,495724090961,46,433,NaN,NaN,NaN,NaN,NaN,127.0.0.1,NaN


In [78]:
import requests
def get_exchange_rates():
    with open('token.txt', 'r') as file:
        api_key = file.read().strip()
    url = f'http://data.fixer.io/api/latest?access_key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'rates' in data:
        rates = data['rates']
        rates[data['base']] = 1.0
        for d in rates:
            rates[d] = 1.0/rates[d]
        return rates
    
rates = get_exchange_rates()

data['channel_usd_value'] = data['Channel Currency'].map(rates) * data['Channel Amount']


,Provider ID,Operation ID,Payment ID,Operation Type,Operation Status,Secure 3ds Check,Channel Amount,Channel Currency,Operation created at Date,Payment Method Type,...,Merchant ID,Project ID,Customer ID,Customer Email,Card Holder,Issuer Country Name,Country By IP Name,Customer IP V4 V6,Issuer Bank Name,channel_usd_value
0,4781.0,99892010097211,6021093719,sale,success,1.0,1000.0,RUB,2022-03-30,visa,...,39,321,3753071,avtomr@yandex.ru,LEONID PROKHOROV,Russian Federation,Russian Federation,176.59.134.184,SBERBANK of Russia,9.554105
1,1293.0,99888010096111,347788572,sale,success,NaN,51.0,EUR,2022-02-27,visa,...,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S,51.000000
2,1293.0,99888010096101,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,...,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S,51.000000
3,1293.0,99888010096091,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,...,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S,51.000000
4,1293.0,99888010096081,347788572,sale,decline,NaN,51.0,EUR,2022-02-27,visa,...,1581,5771,95022,williahmalexdelaplaza@gmail.com,WILLIAHM ALEX DE LA PLAZA,Denmark,Denmark,62.198.132.139,Jyske Bank A/S,51.000000


In [22]:
data_decline = data[data['Operation Status'] == 'decline']

In [121]:
agg_data = data.groupby(['Customer ID', 'Merchant ID']).agg(
    total=('Customer ID', 'size'),
    total_decline=('Operation Status', lambda x: (x == 'decline').sum()),
    unique_IPs=('Customer IP V4 V6', 'nunique'),
    unique_acc_number=('Account Number', 'nunique'),
    unique_countries=('Country By IP Name', 'nunique'),
    unique_merchants=('Merchant ID', 'nunique'),
    unique_cardholders=('Card Holder', 'nunique'),
    unique_emails=('Customer Email', 'nunique'),
    unique_projects=('Project ID', 'nunique'),
    total_country_missmatch=('Country By IP Name', lambda x: (x != data.loc[x.index, 'Issuer Country Name']).sum())
).reset_index()
agg_data = agg_data.sort_values(by='unique_IPs', ascending=False)


agg_data.head()


,Customer ID,Merchant ID,total,total_decline,unique_IPs,unique_acc_number,unique_countries,unique_merchants,unique_cardholders,unique_emails,unique_projects,total_country_missmatch
2055,348231,46,71,16,38,1,2,1,1,1,1,62
323,124221,1581,66,59,17,1,2,1,1,1,1,26
1903,317704,46,20,4,16,1,2,1,1,1,1,18
1966,337528,46,24,21,16,4,1,1,2,1,1,3
2006,344501,46,93,34,16,2,2,1,1,1,1,4


In [126]:
card_usage = data.groupby('Account Number')['Customer ID'].nunique().reset_index()
card_usage.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)

# Идентификация карт, используемых несколькими клиентами
fraudulent_cards = card_usage[card_usage['Customer Count'] > 1]

fraudulent_cards


,Account Number,Customer Count
1,'+79192126910,2
407,409858******9159,2
409,409858******9323,2
490,412720******3539,2
495,412720******4466,2
...,...,...
4571,558306******0424,2
4599,676867******3691,2
4665,77056207258,2
4853,77718263731,2


### Все фродовые айпи адреса

In [241]:
data_fraud = [data_fraud1, data_fraud2, data_fraud3]
fraud = pd.concat(data_fraud)['Customer ID']
fraud

35                        8ba8aa0ac77133680329648aaad0109c
59                        8ba8aa0ac77133680329648aaad0109c
95       523927104ac252a76d599ac2c4a5eeb3d0855d6a7a8664...
188      f8b710cfb991a11a883c0f376fe610c1d4d38750ad8154...
204                       8ba8aa0ac77133680329648aaad0109c
                               ...                        
13535    21af7e457c4a882da5fc577226445ac9c6dca700f0e978...
13552    80adaa3dc01dc25c3bcaaa3543a4054ff2821e8d334eba...
13563    5324ac6b2c99504b3011b61e4bfc9444224d810140b2c9...
13564    a8883369c822ec42b9ed804d331c358eb4ba5adb12d198...
13572    c94bbbccb17e1da80d27e9d2897146a8d3b0927d4dee4f...
Name: Customer ID, Length: 1030, dtype: object